<a href="https://colab.research.google.com/github/adataschultz/MaritimeTrade_COVID/blob/main/MLP_Regression/MaritimeTradeCOVID_MLP_Train2010_2019_batch64_Epochs50_HPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################################
#### 2010 - 2019 Maritime Trade with COVID - MLP #####
##########  Hyperparameter Optimization ##############
######################################################

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Data_MaritimeTrade_MLP/

/content/drive/MyDrive/Data_MaritimeTrade_MLP


In [ ]:
# Install and import packages
!pip install category_encoders
!pip install keras-tuner
from tensorflow import keras
import keras_tuner
from keras_tuner import BayesianOptimization
import os
import random
import pandas as pd
import numpy as np
import category_encoders as ce
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import datetime
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib as plt
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('\n')
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version: {}".format(tf.keras.__version__))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

     |████████████████████████████████| 82 kB 391 kB/s 
     |████████████████████████████████| 97 kB 3.3 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm




TensorFlow version: 2.6.0
Eager execution is: True
Keras version: 2.6.0
Num GPUs Available:  1


In [ ]:
# Reproducibility in TF and Pytorch 
def init_seeds(seed=1019):
    np.random.seed(seed)
    random.seed(seed)
    session_conf = tf.compat.v1.ConfigProto()
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
    os.environ['TF_CUDNN_DETERMINISTIC'] ='true'
    os.environ['TF_DETERMINISTIC_OPS'] = 'true'
    tf.random.set_seed(seed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    return sess

In [ ]:
init_seeds(seed=1019)

In [ ]:
# Needed to reduce dimensions and remove missing data due to memory constraints
df = pd.read_csv('MaritimeTrade_10-19_MLP.csv', low_memory=False)
print('Number of rows and columns:', df.shape)

Number of rows and columns: (26088280, 13)


In [ ]:
# Examine Variables for Missingness
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    var_type = df.dtypes
    mis_val_table = pd.concat([mis_val, mis_val_percent, var_type], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values', 2: 'Data Type'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    print('The selected dataframe has ' + str(df.shape[1]) + ' columns.\n'
          'There are ' + str(mis_val_table_ren_columns.shape[0]) +
          ' columns that have missing values.')
    return mis_val_table_ren_columns

print('\nInitial Missing Data Report') 
print(missing_values_table(df))


Initial Missing Data Report
The selected dataframe has 13 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values, Data Type]
Index: []


In [ ]:
# Examine for other variables unable to impute probalistically
def data_type_quality_table(df):
        var_type = df.dtypes
        unique_count = df.nunique()
        val_table = pd.concat([var_type, unique_count], axis=1)
        val_table_ren_columns = val_table.rename(
        columns = {0 : 'Data Type', 1 : 'Number Unique'})
        print ('The selected dataframe has ' + str(df.shape[1]) + ' columns.\n')
        return val_table_ren_columns

print('\nData Type & Uniqueness Report') 
print(data_type_quality_table(df))


Data Type & Uniqueness Report
The selected dataframe has 13 columns.

                       Data Type  Number Unique
Container_LCL/FCL         object              2
Metric_Tons              float64         151659
Container_Type_Dry          bool              2
TCVUSD                   float64        9784472
Trade_Direction           object              2
Year                       int64             10
us_company_size           object              6
foreign_company_size      object              6
US_Port_Coastal_Region    object              6
Foreign_Country_Region    object             11
HS_Group_Name             object              6
US_Unemployment_Rate     float64             53
Average_Tariff           float64           1583


In [ ]:
# Reformat for X,y
df2 = df.drop(['Metric_Tons'], axis=1)
df1 = df.loc[:, ['Metric_Tons']]
df = pd.concat([df1, df2], axis=1)

del df1, df2

In [ ]:
# X,y for encoding
X = df.drop(['Metric_Tons'], axis=1)
y = df.iloc[:,:1]
print(X.shape)
print(y.shape)

(26088280, 12)
(26088280, 1)


In [ ]:
# Set up train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify= X.Year, random_state = 1019)

In [ ]:
# Training set: Encode variables using ranking - ordinal               
ce_ord = ce.OrdinalEncoder(cols = ['foreign_company_size', 'us_company_size'])
X_train = ce_ord.fit_transform(X_train, y_train['Metric_Tons'])

In [ ]:
# Testing set: Encode variables using ranking - ordinal               
ce_ord = ce.OrdinalEncoder(cols = ['foreign_company_size', 'us_company_size'])
X_test = ce_ord.fit_transform(X_test, y_test['Metric_Tons'])

In [ ]:
# Train: Create dummy variables for categorical variables
X_train = pd.get_dummies(X_train, drop_first=True)
X_train.head()

,Container_Type_Dry,TCVUSD,Year,us_company_size,foreign_company_size,US_Unemployment_Rate,Average_Tariff,Container_LCL/FCL_LCL,Trade_Direction_Import,US_Port_Coastal_Region_Northeast,US_Port_Coastal_Region_Northwest,US_Port_Coastal_Region_Other,US_Port_Coastal_Region_Southeast,US_Port_Coastal_Region_Southwest,Foreign_Country_Region_European Union,Foreign_Country_Region_Middle East & North Africa,Foreign_Country_Region_North America,Foreign_Country_Region_Oceania,Foreign_Country_Region_Other East Asia (not China),Foreign_Country_Region_Other Europe (not European Union),Foreign_Country_Region_South America,Foreign_Country_Region_South Asia,Foreign_Country_Region_Southeast Asia,Foreign_Country_Region_Sub Saharan Africa,HS_Group_Name_Edible with Processing,HS_Group_Name_Finished Goods,HS_Group_Name_Pharma,HS_Group_Name_Raw Input,HS_Group_Name_Vices
10469850,True,38532.89,2014,1,1,6.2,7.29,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
18182491,True,22153.32,2018,2,2,4.0,4.80,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
20616234,True,44206.83,2019,3,3,3.6,17.81,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2749909,True,0.00,2012,2,1,8.3,0.15,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
16605327,True,14485.40,2019,1,3,3.7,24.40,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [ ]:
print('Dimensions of X_train for input:', X_train.shape[1])

Dimensions of X_train for input: 29


In [ ]:
# Test: Create dummy variables for categorical variables
X_test = pd.get_dummies(X_test, drop_first=True)
X_test.head()

,Container_Type_Dry,TCVUSD,Year,us_company_size,foreign_company_size,US_Unemployment_Rate,Average_Tariff,Container_LCL/FCL_LCL,Trade_Direction_Import,US_Port_Coastal_Region_Northeast,US_Port_Coastal_Region_Northwest,US_Port_Coastal_Region_Other,US_Port_Coastal_Region_Southeast,US_Port_Coastal_Region_Southwest,Foreign_Country_Region_European Union,Foreign_Country_Region_Middle East & North Africa,Foreign_Country_Region_North America,Foreign_Country_Region_Oceania,Foreign_Country_Region_Other East Asia (not China),Foreign_Country_Region_Other Europe (not European Union),Foreign_Country_Region_South America,Foreign_Country_Region_South Asia,Foreign_Country_Region_Southeast Asia,Foreign_Country_Region_Sub Saharan Africa,HS_Group_Name_Edible with Processing,HS_Group_Name_Finished Goods,HS_Group_Name_Pharma,HS_Group_Name_Raw Input,HS_Group_Name_Vices
21067397,False,22924.62,2018,1,1,3.8,13.24,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
23343492,True,12584.06,2013,2,2,7.7,16.03,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3164049,True,9539.07,2014,1,3,6.6,7.29,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
18037587,True,147823.99,2016,1,4,5.0,19.44,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
18234872,True,124340.94,2018,3,3,4.0,0.00,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
# MinMax Scaling
mn = MinMaxScaler()
X_train = pd.DataFrame(mn.fit_transform(X_train))
X_test = pd.DataFrame(mn.fit_transform(X_test))

In [ ]:
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#%load_ext tensorboard

In [ ]:
# Set up callbacks
filepath = 'MLP_weights_only_train1019_b16_HPO1.h5'

checkpoint_dir = os.path.dirname(filepath)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_folder,
                                                      histogram_freq=1)

callbacks_list = [EarlyStopping(monitor='val_loss', patience = 5),
                  ModelCheckpoint(filepath, monitor='mse', 
                                  save_best_only = True, mode='min'), 
                  tensorboard_callback]

In [ ]:
# Define model for HPO
def build_model(hp): 
  model = keras.Sequential()
  for i in range(hp.Int("num_layers", 2, 5)):
        model.add(tf.keras.layers.Dense( units=hp.Int("layer_size" + str(i), min_value=20, max_value=40, step=5),
                activation="relu", kernel_initializer='normal'))
  model.add(tf.keras.layers.Dropout(0.3))
  model.add(Dense(1))
  model.compile(loss="mae", metrics=["mse"], optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])))
  return model

In [ ]:
# Define the search conditions 
tuner = BayesianOptimization(
    build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=3,
    overwrite=True,
    directory="MLP_1019_HPO1",
    project_name="MLP_1019_HPO1"
)

In [ ]:
# Print a summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
layer_size0 (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 40, 'step': 5, 'sampling': None}
layer_size1 (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 40, 'step': 5, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
# Begin the search for the best hyperparameters 
tuner.search(X_train, y_train, epochs=3, validation_split=0.2, batch_size=64,
             callbacks=callbacks_list)

Trial 11 Complete [01h 56m 21s]
val_loss: 33.90672047932943

Best val_loss So Far: 33.789422353108726
Total elapsed time: 17h 43m 57s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Retrieve the best model(s)
models = tuner.get_best_models(num_models=2)

In [ ]:
# Print a summary of the results from the trials
tuner.results_summary()

Results summary
Results in MLP_1019_HPO1/MLP_1019_HPO1
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 2
layer_size0: 40
layer_size1: 20
learning_rate: 0.01
layer_size2: 40
layer_size3: 40
layer_size4: 40
Score: 33.789422353108726
Trial summary
Hyperparameters:
num_layers: 3
layer_size0: 40
layer_size1: 25
learning_rate: 0.01
layer_size2: 30
layer_size3: 40
Score: 33.7942746480306
Trial summary
Hyperparameters:
num_layers: 2
layer_size0: 40
layer_size1: 35
learning_rate: 0.01
layer_size2: 20
layer_size3: 40
layer_size4: 40
Score: 33.808152516682945
Trial summary
Hyperparameters:
num_layers: 5
layer_size0: 40
layer_size1: 20
learning_rate: 0.01
layer_size2: 20
layer_size3: 40
layer_size4: 40
Score: 33.89190165201823
Trial summary
Hyperparameters:
num_layers: 2
layer_size0: 40
layer_size1: 20
learning_rate: 0.01
layer_size2: 20
layer_size3: 20
layer_size4: 35
Score: 33.90398152669271
Trial summary
Hyperparameters:
num_layers: 